### Registramos en mlflow

In [ ]:
# Registrar en MLflow
mlflow.sklearn.log_model(model,artifact_path="modelo_Estudiantes") # Mejor Modelo para prod

2025/05/29 19:47:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.




---
### Ahora ponemos en producción la API


In [ ]:
run_id = mlflow.last_active_run().info.run_id
model_uri = f"runs:/{run_id}/modelo_Estudiantes"
print("Model URI:", model_uri)

Model URI: runs:/35364e4131094b159690b5fabd7a3134/modelo_Estudiantes


In [ ]:
! pip install flask_ngrok

In [ ]:
!fuser -k 1234/tcp || true

In [ ]:
!ls /content/drive/MyDrive/mlflow_runs/110792071088175407/$run_id/artifacts/modelo_Estudiantes  # Vemos que el modelo esta dentro de nuestro drive.

conda.yaml  MLmodel  model.pkl	python_env.yaml  requirements.txt


### Ahora que sabemos que nuestro modelo esta y conocemos su ubicación:

In [ ]:
import mlflow.pyfunc

# Ruta al modelo (ajustar al modelo real entrenado)
model_path = f"/content/drive/MyDrive/mlflow_runs/110792071088175407/{run_id}/artifacts/modelo_Estudiantes"
model = mlflow.pyfunc.load_model(model_path) #carga del modelo

### Creamos la Api

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route("/predict", methods=["POST"])
def predict():
    try:
        data = request.get_json()
        df = pd.DataFrame(data['data'], columns=data['columns'])

        pred = model.predict(df)
        return jsonify({"prediction": pred.tolist()})
    except Exception as e:
        return jsonify({"error": str(e)})

In [ ]:
from pyngrok import ngrok
import threading

# Crear un túnel a la API Flask
public_url = ngrok.connect(5002)
print(f"🚀 Tu API Flask está disponible en: {public_url}/predict")

# Ejecutar Flask en segundo plano
def run_flask():
    app.run(port=5002)

thread = threading.Thread(target=run_flask)
thread.start()

🚀 Tu API Flask está disponible en: NgrokTunnel: "https://aa83-34-21-56-251.ngrok-free.app" -> "http://localhost:5002"/predict


### Los datos deben enviarse en el mismo orden con el que fueron entrenados (obviamente sin exam_score):
['age', 'study_hours_per_day', 'social_media_hours', 'netflix_hours', 'attendance_percentage', 'sleep_hours', 'exercise_frequency', 'mental_health_rating', 'exam_score', 'gender_Female', 'gender_Male', 'gender_Other', 'part_time_job_No', 'part_time_job_Yes', 'diet_quality_Fair', 'diet_quality_Good', 'diet_quality_Poor', 'parental_education_level_Bachelor', 'parental_education_level_High School', 'parental_education_level_Master', 'internet_quality_Average', 'internet_quality_Good', 'internet_quality_Poor', 'extracurricular_participation_No', 'extracurricular_participation_Yes']

In [ ]:
lista = df_encoded.head(2).values.tolist()
print(lista)

[[23, 0, 1, 1, 85, 8, 6, 8, 56, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1], [20, 6, 2, 2, 97, 4, 6, 8, 100, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0]]


In [ ]:
import requests

sample = {
  "columns": [
    'age', 'study_hours_per_day', 'social_media_hours', 'netflix_hours', 'attendance_percentage', 'sleep_hours',
    'exercise_frequency', 'mental_health_rating', 'gender_Female', 'gender_Male', 'gender_Other', 'part_time_job_No',
    'part_time_job_Yes', 'diet_quality_Fair', 'diet_quality_Good', 'diet_quality_Poor', 'parental_education_level_Bachelor', 'parental_education_level_High School',
    'parental_education_level_Master', 'internet_quality_Average', 'internet_quality_Good', 'internet_quality_Poor', 'extracurricular_participation_No', 'extracurricular_participation_Yes'
  ],
  "data": [
    [23, 0, 1, 1, 85, 8, 6, 8, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1], # Nota les borre a mano el exam score
    [20, 6, 2, 2, 97, 4, 6, 8, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0]
  ]
}
response = requests.post("https://aa83-34-21-56-251.ngrok-free.app/predict", json=sample)
print("Status code:", response.status_code)
print("Response text:", response.text)  # Esto te muestra qué devolvió el servidor (aunque no sea JSON)
try:
    data = response.json()
    print("JSON response:", data)
except Exception as e:
    print("Error decodificando JSON:", e)

INFO:werkzeug:127.0.0.1 - - [29/May/2025 20:58:07] "POST /predict HTTP/1.1" 200 -


Status code: 200
Response text: {"prediction":[56.60895414159046,101.74224075523848]}

JSON response: {'prediction': [56.60895414159046, 101.74224075523848]}


### Si nos fijamos el exam_score de los primeros 2 estudiantes:

In [ ]:
print(df_encoded['exam_score'].head(2))

0     56
1    100
Name: exam_score, dtype: int64


### La prediccion del mejor Modelo propuesto por Pycaret que es Bayesian Ridge nos dice que el primer alumno obtendrá un 56,6 de nota basado en sus habitos y el segundo alumno un 101,7 lo cual es muy certero pero tambien vemos que se va de rango. Esto puede deberse a:

* El modelo no está restringido a ese rango.

* Los datos de entrenamiento incluían valores atípicos.

* O no normalizamos adecuadamente ni restringimos el output.